In [1]:
# Note that this code uses libraries from the TENNLab Neuromorphic Framework
import eons
import neuro
import risp
from torchvision import datasets, transforms
from random import randint, choice
import json

# some values
NUM_INPUTS = 784
NUM_OUTPUTS = 10
NUM_HIDDEN = 200
NUM_SYNAPSES = 1000
NUM_NEURONS = NUM_INPUTS+NUM_OUTPUTS+NUM_HIDDEN 
MOA = neuro.MOA()
MOA.seed(132312, '') # NEED THESE OTHERWISE DOESN'T RANDOMIZE

DATASET_CAP = 200 # for debugging

# eons and encoding stuff
POP_SIZE = 75
RATE_ENCODING_INTERVAL = 20

In [2]:
# Fashion MNIST dataset
transform = transforms.ToTensor() # scale down to 0-1

train_dataset = datasets.FashionMNIST(root='./data', train=True, transform=transform, download=True)
test_dataset = datasets.FashionMNIST(root='./data', train=False, transform=transform, download=True)

In [3]:
# preprocessing
train_dataset = [(train_dataset[i][0].flatten().tolist(), train_dataset[i][1]) for i in range(len(train_dataset))][:DATASET_CAP]
test_dataset = [(test_dataset[i][0].flatten().tolist(), test_dataset[i][1]) for i in range(len(test_dataset))]

In [4]:
risp_config = { #https://bitbucket.org/neuromorphic-utk/framework/src/93a1bbba2c0a31ca8f036ff2a89a4d6ffae5cec3/processors/risp/README.md#markdown-header-params
  "min_weight": -1,
  "max_weight": 1,
  "min_threshold": -1,
  "max_threshold": 1,
  "min_potential": -1,
  "max_delay": 10,
  "discrete": False
}

proc = risp.Processor(risp_config)

# configure eons
eons_param = {
    "starting_nodes": NUM_HIDDEN,
    "starting_edges": NUM_SYNAPSES,
    "merge_rate": 0,
    "population_size": POP_SIZE,
    "multi_edges": 0,
    "crossover_rate": 0.9,
    "mutation_rate": 0.9,
    "selection_type": "tournament",
    "tournament_size_factor": 0.1,
    "tournament_best_net_factor": 0.9,
    "random_factor": 0.65,
    "num_mutations": 100,
    "node_mutations": { "Threshold": 1.0 },
    "net_mutations": { },
    "edge_mutations": { "Weight": 0.65 , "Delay": 0.35,  },
    "num_best" : 4,
    "add_node_rate": 0.75,
    "delete_node_rate": 0.25,
    "add_edge_rate": 0.75,
    "delete_edge_rate": 0.25,
    "node_params_rate": 2.5,
    "edge_params_rate": 2.5,
    "net_params_rate" : 0
}
eons_inst = eons.EONS(eons_param)

In [5]:
net = neuro.Network()
net.set_properties(proc.get_network_properties())

# create input, output and hidden neurons
input_ids = []
hidden_ids = []
output_ids = []

for i in range(NUM_INPUTS):
    node = net.add_node(i)
    net.randomize_node_properties(MOA, node)
    node.set("Threshold", 1)
    net.add_input(i)
    input_ids.append(i)

for i in range(NUM_INPUTS, NUM_INPUTS+NUM_OUTPUTS):
    node = net.add_node(i)
    net.randomize_node_properties(MOA, node)
    node.set("Threshold", 1)
    net.add_output(i)
    output_ids.append(i)

for i in range(NUM_INPUTS+NUM_OUTPUTS, NUM_INPUTS+NUM_OUTPUTS+NUM_HIDDEN):
    node = net.add_node(i)
    net.randomize_node_properties(MOA, node)
    hidden_ids.append(i)

# create synapses between input and hidden
for i in range(NUM_SYNAPSES//2):
    synapse = net.add_or_get_edge(choice(input_ids), choice(hidden_ids))
    net.randomize_edge_properties(MOA, synapse)

# create synapses between hidden and output
for i in range(NUM_SYNAPSES//2):
    synapse = net.add_or_get_edge(choice(hidden_ids), choice(output_ids))
    net.randomize_edge_properties(MOA, synapse)

In [6]:
with open('dump.json', 'w') as f:
    f.write(str(net))

In [6]:
# Set up encoder, we'll use rate encoding

settings_encoder = {
    'dmin': [0 for i in range(784)],
    'dmax': [1 for i in range(784)],
    'default_interval': RATE_ENCODING_INTERVAL,
    "encoders": [ "rate" ]
}

encoder = neuro.EncoderArray(settings_encoder)

# debug - looks like encoder array automatically creates our spikes at the given rate for each id
# I'm assuming it just guesses input ids based on element of our input list index?
spikes = encoder.get_spikes(test_dataset[0][0])
for s in spikes:
    pass
    #print(f'({s.id} at {s.time:.2f})', end=', ')

In [7]:
# Apply spikes as an example
proc.load_network(net)

for i in range(net.num_nodes()):
    proc.track_neuron_events(i)

proc.apply_spikes(spikes)

proc.run(500)
v = proc.neuron_vectors()
for i in range(len(v)):
    if len(v[i]) != 0:
        print(f'{i} fires at {v[i]}')

#proc.neuron_last_fires()

215 fires at [0.0]
216 fires at [0.0]
219 fires at [0.0]
221 fires at [0.0, 6.0, 13.0]
237 fires at [0.0]
238 fires at [0.0]
240 fires at [0.0, 9.0, 18.0]
241 fires at [0.0, 3.0, 6.0, 9.0, 12.0, 15.0, 18.0]
242 fires at [0.0]
249 fires at [0.0, 2.0, 4.0, 6.0, 8.0, 10.0, 12.0, 14.0, 17.0, 19.0]
265 fires at [0.0]
268 fires at [0.0, 2.0, 5.0, 8.0, 11.0, 14.0, 17.0]
269 fires at [0.0, 1.0, 3.0, 5.0, 7.0, 8.0, 10.0, 12.0, 14.0, 16.0, 17.0, 19.0]
270 fires at [0.0, 2.0, 4.0, 6.0, 9.0, 11.0, 13.0, 16.0, 18.0]
275 fires at [0.0, 11.0]
276 fires at [0.0, 2.0, 5.0, 8.0, 10.0, 13.0, 16.0, 19.0]
277 fires at [0.0, 2.0, 4.0, 7.0, 9.0, 12.0, 14.0, 16.0, 19.0]
293 fires at [0.0]
295 fires at [0.0, 4.0, 9.0, 14.0, 19.0]
296 fires at [0.0, 1.0, 3.0, 5.0, 7.0, 9.0, 11.0, 13.0, 15.0, 17.0, 19.0]
297 fires at [0.0, 2.0, 4.0, 6.0, 8.0, 10.0, 12.0, 14.0, 16.0, 19.0]
298 fires at [0.0, 1.0, 3.0, 5.0, 6.0, 8.0, 10.0, 12.0, 13.0, 15.0, 17.0, 19.0]
299 fires at [0.0, 1.0, 2.0, 4.0, 5.0, 7.0, 8.0, 10.0, 11.0, 1

In [8]:
print(dir(proc))
proc.output_counts()

['__class__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '_pybind11_conduit_v1_', 'apply_binary_data', 'apply_dvs_events', 'apply_spike', 'apply_spikes', 'clear', 'clear_activity', 'get_name', 'get_network_properties', 'get_params', 'get_processor_properties', 'get_time', 'load_network', 'load_networks', 'neuron_charges', 'neuron_counts', 'neuron_last_fires', 'neuron_vectors', 'output_count', 'output_count_all', 'output_count_max', 'output_counts', 'output_last_fire', 'output_last_fires', 'output_vector', 'output_vectors', 'run', 'synapse_weights', 'total_neuron_accumulates', 'total_neuron_counts', 'track_neuron_events', 'track_output_events']


[0, 0, 1, 3, 4, 12, 25, 6, 15, 5]

In [9]:
# some functions for viewing information to debug
def network_details(nw):
    #print(dir(nw))
    net_json = nw.as_json()
    print(f'Network has {len(net_json["Edges"])} synapses and {len(net_json["Nodes"])} neurons')

    # check if all out nodes have an incoming synapse
    out_ids = net_json['Outputs']
    for edge in net_json['Edges']:
        if edge['to'] in out_ids:
            out_ids.remove(edge['to'])

    if len(out_ids) == 0:
        print('All outputs have incoming connections')
    else:
        print(f'Outputs {out_ids} have no incoming connections')


# also count classes in training set
class_count = {i:0 for i in range(0, 10)}
for t in train_dataset:
    class_count[t[1]] += 1

print(class_count)

network_details(net)

{0: 24, 1: 26, 2: 18, 3: 17, 4: 18, 5: 20, 6: 21, 7: 21, 8: 16, 9: 19}
Network has 940 synapses and 994 neurons
All outputs have incoming connections


In [10]:
# eons training
eons_inst.set_template_network(net) # strips hidden and edges, fixes input/outputs
pop = eons_inst.generate_population(eons_param, MOA.random_integer())

#for n in pop.networks:
    #print(n.network.num_edges())

# for now let's say our fitness is # of correct predictions on training set
def compute_fitness(nw, debug=False):
    proc.load_network(nw)
    #network_details(nw)

    correct = 0
    total = 0
    class_correct = {i:0 for i in range(0, 10)}
    for sample in train_dataset:
        proc.clear_activity()
        spikes = encoder.get_spikes(sample[0])
        proc.apply_spikes(spikes)
        proc.run(2000)
        out_counts = proc.output_counts()

        prediction = out_counts.index(max(out_counts))
        if prediction == sample[1]:
            correct += 1

            if debug: class_correct[sample[1]] += 1
        total += 1

    if debug: # per class accuracy 
        for i in range(0, 10):
            print(f'Class {i} - {(class_correct[i]/class_count[i])*100:.2f}%')
        
    return correct

# OKAY, maybe guaranteeing that it gets things correct across all classes should be a part of fitness
    

for i in range(100):
    print(f'STARTING EPOCH {i}')
    fits = [compute_fitness(n.network) for n in pop.networks]
    print(f'best accuracy {max(fits)/DATASET_CAP}')
    print('Details on best network:')
    best = pop.networks[fits.index(max(fits))].network
    network_details(best)
    
    #print(f'Average accuracy of pop {(sum(fits)/len(fits))/DATASET_CAP:.2f}')
    pop = eons_inst.do_epoch(pop, fits, eons_param)

STARTING EPOCH 0
best accuracy 0.165
Details on best network:
Network has 1000 synapses and 994 neurons
Outputs [786, 791] have no incoming connections
STARTING EPOCH 1
best accuracy 0.215
Details on best network:
Network has 962 synapses and 1002 neurons
Outputs [786, 788, 789, 791] have no incoming connections
STARTING EPOCH 2
best accuracy 0.26
Details on best network:
Network has 968 synapses and 1001 neurons
Outputs [788, 789, 792] have no incoming connections
STARTING EPOCH 3
best accuracy 0.26
Details on best network:
Network has 968 synapses and 1001 neurons
Outputs [788, 789, 792] have no incoming connections
STARTING EPOCH 4
best accuracy 0.285
Details on best network:
Network has 977 synapses and 1012 neurons
Outputs [789] have no incoming connections
STARTING EPOCH 5
best accuracy 0.285
Details on best network:
Network has 977 synapses and 1012 neurons
Outputs [789] have no incoming connections
STARTING EPOCH 6
best accuracy 0.285
Details on best network:
Network has 977 sy

In [11]:
fits = [compute_fitness(n.network) for n in pop.networks]
print(f'best accuracy {max(fits)/DATASET_CAP}')
print('Details on best network:')
n = pop.networks[fits.index(max(fits))].network
network_details(n)

best accuracy 0.41
Details on best network:
Network has 1174 synapses and 1324 neurons
Outputs [784, 786] have no incoming connections


In [12]:
compute_fitness(n, debug=True)

Class 0 - 4.17%
Class 1 - 96.15%
Class 2 - 0.00%
Class 3 - 0.00%
Class 4 - 44.44%
Class 5 - 50.00%
Class 6 - 80.95%
Class 7 - 100.00%
Class 8 - 0.00%
Class 9 - 0.00%


82